In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf

config = SparkConf()
config.set("spark.app.name", "First Spark RDD app")

sc = SparkContext(conf=config)

In [3]:
# ! hdfs dfs -ls /labs/laba01/ml-100k

In [4]:
# Файл u.data содержит все оценки, а файл u.item — список всех фильмов.
# id фильма для расчета индивидуальных характеристик — в Личном кабинете на странице Лабы 1.
# film id for calculations
# 302
# В поле “hist_film” нужно указать для заданного id фильма количество поставленных оценок в следующем порядке:
#                 "1", "2", "3", "4", "5". То есть, сколько было единичек, двоек, троек и т.д.
# В поле “hist_all” нужно указать то же самое, только для всех фильмов общее количество поставленных 
#               оценок в том же порядке: "1", "2", "3", "4", "5".

In [5]:
# !hdfs dfs -cat /labs/laba01/ml-100k/u.data

In [6]:
# !hdfs dfs -cat /labs/laba01/ml-100k/u.item

In [7]:
rdd2 = sc.textFile("/labs/laba01/ml-100k/u.data")

In [8]:
rdd2.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [9]:
rdd2 = rdd2.map(lambda x: x.split("\t"))

In [10]:
rdd2.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [11]:
rdd302 = rdd2.filter(lambda x: x[1] == "302")

In [12]:
rdd302.take(5)

[['186', '302', '3', '891717742'],
 ['191', '302', '4', '891560253'],
 ['49', '302', '4', '888065432'],
 ['54', '302', '4', '880928519'],
 ['62', '302', '3', '879371909']]

In [13]:
csi302 = rdd302.flatMap(lambda x: x[2])
csi = rdd2.flatMap(lambda x: x[2])

In [14]:
csi.take(10)

['3', '3', '1', '2', '1', '4', '2', '5', '3', '3']

In [15]:
csi302g = csi302.groupBy(lambda x: x)

In [16]:
csi302g.take(10)

[('4', <pyspark.resultiterable.ResultIterable at 0x7ff3084bc4a8>),
 ('1', <pyspark.resultiterable.ResultIterable at 0x7ff3084bc940>),
 ('3', <pyspark.resultiterable.ResultIterable at 0x7ff308467ac8>),
 ('5', <pyspark.resultiterable.ResultIterable at 0x7ff308467978>),
 ('2', <pyspark.resultiterable.ResultIterable at 0x7ff308467b38>)]

In [17]:
csi302g = (csi302.countByKey().items() )

In [18]:
csi302g

dict_items([('3', 46), ('4', 119), ('5', 120), ('2', 10), ('1', 2)])

In [19]:
csig = (csi.countByKey().items() )

In [20]:
csig

dict_items([('3', 27145), ('1', 6110), ('2', 11370), ('4', 34174), ('5', 21201)])

In [21]:
d_302 = dict(csi302g)
d_all = dict(csig)

In [22]:
d_302

{'3': 46, '4': 119, '5': 120, '2': 10, '1': 2}

In [23]:
res = dict()
res["hist_film"] = [d_302['1'], d_302['2'], d_302['3'], d_302['4'], d_302['5']]
res["hist_all"] = [d_all['1'], d_all['2'], d_all['3'], d_all['4'], d_all['5']]

In [24]:
res

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [25]:
import json

In [26]:
with open('lab01.json', 'w') as f:
    json.dump(res, f, indent=3)

In [27]:
sc.stop()

In [28]:
print(res)

{'hist_film': [2, 10, 46, 119, 120], 'hist_all': [6110, 11370, 27145, 34174, 21201]}
